In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPModel
import numpy as np
import torch
import rp
from transformers import CLIPProcessor, CLIPModel

from source.clip import get_clip_logits
import source.peekaboo as peekaboo

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
url = "https://i.pinimg.com/564x/87/a1/3e/87a13ebe6b5e4f100b84b0e68bc994d5--pokemon-cakes-pokemon-go.jpg"
image=rp.as_float_image(rp.as_rgb_image(rp.load_image(url)))
image=rp.as_torch_image(image)
image=image.to('cuda:2')

logits_per_image = get_clip_logits(image,["a photo of a squirtle","a photo of a pokemon","a photo of a pikachu",""])
probs = logits_per_image.softmax(dim=0)  # we can take the softmax to get the label probabilities
print(logits_per_image,probs)

rp.display_image(rp.load_image(url))

In [ ]:
import icecream
import numpy as np
import rp
import torch
import torch.nn as nn
from easydict import EasyDict
from IPython.display import clear_output
from torchvision.transforms.functional import normalize

from source.bilateral_blur import BilateralProxyBlur
from source.learnable_textures import (LearnableImageFourier,
                                    LearnableLatentImage,
                                       LearnableImageFourierBilateral,
                                       LearnableImageRaster,
                                       LearnableImageRasterBilateral,
                                       LearnableTexturePackFourier,
                                       LearnableTexturePackRaster)

In [ ]:
image = LearnableLatentImage(
    LearnableImageRaster(height=64, width=64, num_channels=4).cuda(),
    lambda x: peekaboo.s.decode_latent((x-.5)*1),
)

# image=LearnableImageFourier(224,224)
# image=LearnableImageRaster(height=64, width=64, num_channels=4).cuda()
    
    

def display():
    rp.display_image(rp.as_numpy_image(image()))
display()

In [ ]:
prompt='a kitten in a cardboard box'
prompt='blue'

optim=torch.optim.Adam(
    image.parameters(),    
    # lr=1e-1
)

for i in range(10000):
    
    logits = get_clip_logits(image(),[prompt, ""])

    loss= - (logits[0]*1 - logits[1])
    loss=loss*1
    loss.backward()
    # break
    optim.step()
    optim.zero_grad()
    
    if not i%25:
        print(i, loss)
        display()

In [ ]:
image()